In [ ]:
# Step 1: Define Default Parameters
EMA_HOST = "169.254.9.108"
CENTRE_FREQ_MHZ = 5950
LNA_EN = True
OFFSET_DB = 156
PREINITIALISE = True
SHOW_PLOTS = True

In [ ]:
# Step 2: Define Functions
import ssh

OCPI_CDK_DIR = "/run/media/mmcblk0p2/opencpi"
OCPI_LOCAL_DIR = OCPI_CDK_DIR
OCPI_ROOT_DIR = OCPI_CDK_DIR + "/.."
OCPI_LIBRARY_PATH = OCPI_CDK_DIR + "/ema_lb_rcc/artifacts:" + OCPI_CDK_DIR + "/artifacts"
OCPI_SYSTEM_CONFIG = OCPI_CDK_DIR + "/system.xml"
OCPI_TOOL_PLATFORM = "ema_lb_rcc"
OCPI_DEFAULT_HDL_DEVICE = "pl:0"
OCPI_TOOL_DIR = OCPI_TOOL_PLATFORM
OCPI_TOOL_OS = "linux"
OCPI_RELEASE = "opencpi-v2.3.0"
APP_ROOT = "/run/media/nvme0n1/opencpi"
APP_REL_DIR = "applications/ocpi_xcvrtool"
APP_PATH = "target-ema_lb_rcc/ocpi_xcvrtool"

passwords = {}
s = ssh.SSH(EMA_HOST, passwords)

ENVIRONMENT = "export OCPI_CDK_DIR=" + OCPI_CDK_DIR
ENVIRONMENT += ";export OCPI_LOCAL_DIR=" + OCPI_LOCAL_DIR
ENVIRONMENT += ";export OCPI_ROOT_DIR=" + OCPI_ROOT_DIR
ENVIRONMENT += ";export OCPI_LIBRARY_PATH=" + OCPI_LIBRARY_PATH
ENVIRONMENT += ";export OCPI_SYSTEM_CONFIG=" + OCPI_SYSTEM_CONFIG
ENVIRONMENT += ";export OCPI_TOOL_PLATFORM=" + OCPI_TOOL_PLATFORM
ENVIRONMENT += ";export OCPI_DEFAULT_HDL_DEVICE=" + OCPI_DEFAULT_HDL_DEVICE
ENVIRONMENT += ";export OCPI_TOOL_DIR=" + OCPI_TOOL_DIR
ENVIRONMENT += ";export OCPI_TOOL_OS=" + OCPI_TOOL_OS
ENVIRONMENT += ";export OCPI_RELEASE=" + OCPI_RELEASE

def preinitialise():
    s.send_command("/bin/echo {}/{} > /tmp/ocpi_app_directory".format(APP_ROOT, APP_REL_DIR))
    s.send_command("cd {};. ./kcema_setup.sh {}".format(OCPI_CDK_DIR, "/tmp")) # Load OpenCPI bitstream
    s.send_command("/sbin/devmem 0x40014000 32 0x00") # Disable all PSUs
    s.send_command("/sbin/devmem 0x40014000 32 0xFF") # Enable all PSUs
    s.send_command("/sbin/devmem 0x40015008 32 0x01") # Enable clock generator
    s.send_command("cd /run/media/mmcblk0p2/test;/usr/bin/python3 ad9528.py") # Initialise clock generator
    initialise(6000, False)
    
def initialise(centre_freq_MHz, lna_en):
    lna = 0
    if lna_en:
        lna = 1
    s.send_command("{};cd {}/{};./{} -F {} -L {} -T 0".format(ENVIRONMENT, OCPI_CDK_DIR, APP_REL_DIR, APP_PATH, str(centre_freq_MHz), lna))

def capture(capture_size, filename):    
    s.send_command("/bin/rm /run/media/nvme0n1/{}".format(filename))
    s.send_command("/usr/bin/fallocate /run/media/nvme0n1/{} -l {}".format(filename, capture_size))
    s.send_command("/bin/echo 1 > /sys/kernel/kcema-driver/reset")
    s.send_command("/bin/echo 0 > /sys/kernel/kcema-driver/reset")
    s.send_command("/bin/echo /run/media/nvme0n1/{} > /sys/kernel/kcema-driver/write_path".format(filename))
    s.send_command("/bin/echo 1 > /sys/kernel/kcema-driver/write_state")

In [ ]:
# Step 3: OpenCPI Initialisation
if PREINITIALISE:
    preinitialise()

In [ ]:
# Step 4: Transceiver Initialisation
print("Centre Frequency = {} MHz, LNA Enabled = {}".format(CENTRE_FREQ_MHZ, LNA_EN))
initialise(CENTRE_FREQ_MHZ, LNA_EN)

In [ ]:
# Step 5: Capture I/Q File to EMA SSD
import time
from datetime import datetime

#capture_size = "262144"  # Size of captured file in bytes (64K samples * 2 bytes per sample * 2 [I/Q])
CAPTURE_SIZE = "1M"
FILENAME = "capture.iq"
capture(CAPTURE_SIZE, FILENAME)
time.sleep(3)

In [ ]:
# Step 6: Transfer I/Q File to PC
print("Transferring to host PC: {}".format(FILENAME))
s.get_file("/run/media/nvme0n1/{}".format(FILENAME), FILENAME)

In [ ]:
# Step 7: Read Samples, Plot Spectrogram
import numpy as np
from numpy.fft import fft, fftshift, fftfreq
import seaborn as sns
import matplotlib.pyplot as plt

CENTRE_FREQ_HZ = CENTRE_FREQ_MHZ * 1e6
FFT_SIZE = 65536
SAMPLE_RATE_SPS = 122.88e6
FILE_OFFSET = 65536 * 4  # Skip 64K samples (2-bytes each, I+Q)

samples = np.fromfile(FILENAME, dtype=np.int16, count=FFT_SIZE*2, offset=FILE_OFFSET)

if CENTRE_FREQ_MHZ <= 5900:
    # Not the LO path, no frequency inversion
    iq = (samples[::2] + 1j*samples[1::2])  # convert to IQIQIQ...
else:
    # LO path, frequency inversion, swap I/Q
    iq = (samples[1::2] + 1j*samples[::2])  # convert to QIQIQI...

if SHOW_PLOTS:
    plt.specgram(iq, Fs=SAMPLE_RATE_SPS, Fc=CENTRE_FREQ_HZ, scale="dB")#, vmin=-210, vmax=-100)
    plt.colorbar()

In [ ]:
# Step 8: Plot FFT
f=fft(iq*np.blackman(FFT_SIZE))
x=fftfreq(FFT_SIZE) * SAMPLE_RATE_SPS / 1e6
spec=20*np.log10(np.abs(f))-OFFSET_DB

sns.set_style('darkgrid')
fig, ax = plt.subplots()
fig.tight_layout(pad=4)
ax.plot(x, spec)
ax.grid(True)
ax.set_xlabel("Frequency Offset (MHz)\nfc = {} MHz, LNA Enabled = {}".format(CENTRE_FREQ_MHZ, LNA_EN))
ax.set_ylabel("Power (dBm)")
ax.set_ylim(-120, 0)
ymax = max(spec)
xpos = np.where(spec == ymax)
xmax = x[xpos]
ax.annotate("{:.2f} MHz, {:.2f} dBm".format(float((CENTRE_FREQ_MHZ)+xmax), ymax), xy=(xmax, ymax), xytext=(xmax, ymax+8), arrowprops=dict(facecolor='black', shrink=0.05))
print("Marker: {:.2f} MHz, {:.2f} dBm".format(float((CENTRE_FREQ_MHZ)+xmax), ymax))
plt.savefig('fft_plot.png')
if SHOW_PLOTS:
    plt.show()